In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Functions")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/09/09 20:04:05 INFO SparkContext: Running Spark version 3.5.0
24/09/09 20:04:05 INFO SparkContext: OS info Linux, 6.8.0-40-generic, amd64
24/09/09 20:04:05 INFO SparkContext: Java version 11.0.24
24/09/09 20:04:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/09 20:04:06 INFO ResourceUtils: ==============================================================
24/09/09 20:04:06 INFO ResourceUtils: No custom resources configured for spark.driver.
24/09/09 20:04:06 INFO ResourceUtils: ==============================================================
24/09/09 20:04:06 INFO SparkContext: Submitted application: Functions
24/09/09 20:04:06 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap ->

spark: SparkSession = org.apache.spark.sql.SparkSession@5dfe391f
import spark.implicits._

In [4]:
val data = Seq(
    ("Jean-Georges", "Perrin", "NC", 1, 300, 1551903533),
    ("Jean-Georges", "Perrin", "NC", 2, 120, 1551903567),
    ("Jean-Georges", "Perrin", "CA" ,4, 75, 1551903599),
    ("Holden", "Karau", "CA" , 6, 37, 1551904299),
    ("Ginni", "Rometty", "NY", 7, 91, 1551916792),
    ("Holden", "Karau", "CA", 4, 153, 1552876129)
).toDF("firstName", "lastName", "state", "quantity", "revenue", "timestamp")

data.show()

24/09/09 20:04:09 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/09/09 20:04:09 INFO SharedState: Warehouse path is 'file:/home/vadim/workspace/Spark/UDF%20UDAF/spark-warehouse'.
24/09/09 20:04:10 INFO CodeGenerator: Code generated in 265.717015 ms
24/09/09 20:04:12 INFO CodeGenerator: Code generated in 12.354238 ms
24/09/09 20:04:12 INFO CodeGenerator: Code generated in 19.293655 ms


+------------+--------+-----+--------+-------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|
+------------+--------+-----+--------+-------+----------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|
|      Holden|   Karau|   CA|       6|     37|1551904299|
|       Ginni| Rometty|   NY|       7|     91|1551916792|
|      Holden|   Karau|   CA|       4|    153|1552876129|
+------------+--------+-----+--------+-------+----------+



data: DataFrame = [firstName: string, lastName: string ... 4 more fields]

In [5]:
data.printSchema()

root
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- state: string (nullable = true)
 |-- quantity: integer (nullable = false)
 |-- revenue: integer (nullable = false)
 |-- timestamp: integer (nullable = false)



## Array Functions

In [6]:
val datak2 = data
                .withColumn("k2v", array($"firstName", $"lastName", $"state"))
                .withColumn("k2n", array(lit("FirstName"), lit("LastName"), lit("State")))

datak2.show(false)

24/09/09 20:04:13 INFO CodeGenerator: Code generated in 10.971148 ms
24/09/09 20:04:13 INFO CodeGenerator: Code generated in 17.347162 ms


+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2v                       |k2n                         |
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[Jean-Georges, Perrin, CA]|[FirstName, LastName, State]|
|Holden      |Karau   |CA   |6       |37     |1551904299|[Holden, Karau, CA]       |[FirstName, LastName, State]|
|Ginni       |Rometty |NY   |7       |91     |1551916792|[Ginni, Rometty, NY]      |[FirstName, LastName, State]|
|Holden      |Karau   |CA   |4       |153    |1552876129|[Holden, Karau, CA]       |[Fir

datak2: DataFrame = [firstName: string, lastName: string ... 6 more fields]

In [7]:
datak2.where(array_contains($"k2v", "CA")).show(false)

24/09/09 20:04:14 INFO CodeGenerator: Code generated in 8.273841 ms


+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2v                       |k2n                         |
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[Jean-Georges, Perrin, CA]|[FirstName, LastName, State]|
|Holden      |Karau   |CA   |6       |37     |1551904299|[Holden, Karau, CA]       |[FirstName, LastName, State]|
|Holden      |Karau   |CA   |4       |153    |1552876129|[Holden, Karau, CA]       |[FirstName, LastName, State]|
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+



## Map Functions

In [8]:
val datak2m = data.withColumn("k2m", map(lit("FirstName"), $"firstName", lit("LastName"), $"lastName", lit("State"), $"state"))

datak2m.show(false)

24/09/09 20:04:14 INFO CodeGenerator: Code generated in 9.318766 ms
24/09/09 20:04:14 INFO CodeGenerator: Code generated in 14.418347 ms


+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2m                                                         |
+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|{FirstName -> Jean-Georges, LastName -> Perrin, State -> CA}|
|Holden      |Karau   |CA   |6       |37     |1551904299|{FirstName -> Holden, LastName -> Karau, State -> CA}       |
|Ginni       |Rometty |NY   |7       |91     |1551916792|{FirstName -> Ginni, LastName -> Rometty, State -> NY}      |
|Holden      |Karau   |CA   |4       |153    |15

datak2m: DataFrame = [firstName: string, lastName: string ... 5 more fields]

In [9]:
val datak2ma = datak2.withColumn("k2m", map_from_arrays($"k2n", $"k2v"))

datak2ma.show(false)

24/09/09 20:04:15 INFO CodeGenerator: Code generated in 10.377395 ms
24/09/09 20:04:15 INFO CodeGenerator: Code generated in 15.076361 ms


+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+------------------------------------------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2v                       |k2n                         |k2m                                                         |
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+------------------------------------------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[Jean-Georges, Perrin, CA]|[FirstName, LastName, State]|{FirstName -

datak2ma: DataFrame = [firstName: string, lastName: string ... 7 more fields]

## Date and Timestamp Functions

In [10]:
val datat = data
                .withColumn("recordTimestamp", to_timestamp($"timestamp"))
                .withColumn("current", current_timestamp())

datat.show(false)

+------------+--------+-----+--------+-------+----------+-------------------+--------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |recordTimestamp    |current                   |
+------------+--------+-----+--------+-------+----------+-------------------+--------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|2019-03-06 23:18:53|2024-09-09 20:04:15.764363|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|2019-03-06 23:19:27|2024-09-09 20:04:15.764363|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|2019-03-06 23:19:59|2024-09-09 20:04:15.764363|
|Holden      |Karau   |CA   |6       |37     |1551904299|2019-03-06 23:31:39|2024-09-09 20:04:15.764363|
|Ginni       |Rometty |NY   |7       |91     |1551916792|2019-03-07 02:59:52|2024-09-09 20:04:15.764363|
|Holden      |Karau   |CA   |4       |153    |1552876129|2019-03-18 05:28:49|2024-09-09 20:04:15.764363|
+------------+--------+-----+--------+-------+---------

datat: DataFrame = [firstName: string, lastName: string ... 6 more fields]

In [19]:
datat.printSchema

root
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- state: string (nullable = true)
 |-- quantity: integer (nullable = false)
 |-- revenue: integer (nullable = false)
 |-- timestamp: integer (nullable = false)
 |-- recordTimestamp: timestamp (nullable = false)
 |-- current: timestamp (nullable = false)



In [11]:
datat.select($"recordTimestamp", $"current", datediff($"current", $"recordTimestamp")).show(false)

24/09/09 20:04:16 INFO CodeGenerator: Code generated in 8.259 ms
24/09/09 20:04:16 INFO CodeGenerator: Code generated in 9.496741 ms


+-------------------+--------------------------+----------------------------------+
|recordTimestamp    |current                   |datediff(current, recordTimestamp)|
+-------------------+--------------------------+----------------------------------+
|2019-03-06 23:18:53|2024-09-09 20:04:16.281041|2014                              |
|2019-03-06 23:19:27|2024-09-09 20:04:16.281041|2014                              |
|2019-03-06 23:19:59|2024-09-09 20:04:16.281041|2014                              |
|2019-03-06 23:31:39|2024-09-09 20:04:16.281041|2014                              |
|2019-03-07 02:59:52|2024-09-09 20:04:16.281041|2013                              |
|2019-03-18 05:28:49|2024-09-09 20:04:16.281041|2002                              |
+-------------------+--------------------------+----------------------------------+



## JSON Functions

In [12]:
val datak2j = datak2m.withColumn("k2j", to_json($"k2m"))

datak2j.show(false)

+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+-------------------------------------------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2m                                                         |k2j                                                          |
+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+-------------------------------------------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|{"FirstName":"Jean-Georges","LastName":"Perrin","State":"NC"}|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|{"FirstName":"Jean-Georges","LastName":"Perrin","State":"NC"}|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|{FirstName -> Jean-Georges, LastName -

datak2j: DataFrame = [firstName: string, lastName: string ... 6 more fields]

## Generator Functions

In [13]:
val data2 = data
                .withColumn("a1", array(lit(1), lit(2), lit(3), lit(4), lit(5)))
                .withColumn("a2", lit((1 to 5).toArray))

data2.show(false)

+------------+--------+-----+--------+-------+----------+---------------+---------------+
|firstName   |lastName|state|quantity|revenue|timestamp |a1             |a2             |
+------------+--------+-----+--------+-------+----------+---------------+---------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Holden      |Karau   |CA   |6       |37     |1551904299|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Ginni       |Rometty |NY   |7       |91     |1551916792|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Holden      |Karau   |CA   |4       |153    |1552876129|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
+------------+--------+-----+--------+-------+----------+---------------+---------------+



data2: DataFrame = [firstName: string, lastName: string ... 6 more fields]

In [14]:
data2.withColumn("dummy", explode($"a1")).show(false)

24/09/09 20:04:17 INFO CodeGenerator: Code generated in 5.66023 ms
24/09/09 20:04:18 INFO CodeGenerator: Code generated in 28.554711 ms
24/09/09 20:04:18 INFO CodeGenerator: Code generated in 15.779777 ms
24/09/09 20:04:18 INFO SparkContext: Starting job: show at cell14.sc:1
24/09/09 20:04:18 INFO DAGScheduler: Got job 0 (show at cell14.sc:1) with 1 output partitions
24/09/09 20:04:18 INFO DAGScheduler: Final stage: ResultStage 0 (show at cell14.sc:1)
24/09/09 20:04:18 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:04:18 INFO DAGScheduler: Missing parents: List()
24/09/09 20:04:18 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[2] at show at cell14.sc:1), which has no missing parents
24/09/09 20:04:18 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 13.5 KiB, free 4.5 GiB)
24/09/09 20:04:18 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 5.2 KiB, free 4.5 GiB)
24/09/09 20:04:18 INFO BlockMan

+------------+--------+-----+--------+-------+----------+---------------+---------------+-----+
|firstName   |lastName|state|quantity|revenue|timestamp |a1             |a2             |dummy|
+------------+--------+-----+--------+-------+----------+---------------+---------------+-----+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|1    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|2    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|3    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|4    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|5    |
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|1    |
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|2    |
|Jean-Georges|Perrin  |NC   |2       |12

## Window Functions

In [15]:
import org.apache.spark.sql.expressions.Window

val windowSpec = Window.partitionBy("firstName", "lastName").orderBy("state")

import org.apache.spark.sql.expressions.Window
windowSpec: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2e691295

In [16]:
val dataw = data.withColumn("row_number", row_number().over(windowSpec))

dataw.orderBy("firstName", "lastName", "state").show()

24/09/09 20:04:19 INFO CodeGenerator: Code generated in 7.922256 ms
24/09/09 20:04:19 INFO DAGScheduler: Registering RDD 5 (show at cell16.sc:3) as input to shuffle 0
24/09/09 20:04:19 INFO DAGScheduler: Got map stage job 2 (show at cell16.sc:3) with 6 output partitions
24/09/09 20:04:19 INFO DAGScheduler: Final stage: ShuffleMapStage 2 (show at cell16.sc:3)
24/09/09 20:04:19 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:04:19 INFO DAGScheduler: Missing parents: List()
24/09/09 20:04:19 INFO DAGScheduler: Submitting ShuffleMapStage 2 (MapPartitionsRDD[5] at show at cell16.sc:3), which has no missing parents
24/09/09 20:04:19 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 8.9 KiB, free 4.5 GiB)
24/09/09 20:04:19 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 4.7 KiB, free 4.5 GiB)
24/09/09 20:04:19 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on ubuntu:37259 (size: 4.7 KiB, free: 4.5 GiB)


+------------+--------+-----+--------+-------+----------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_number|
+------------+--------+-----+--------+-------+----------+----------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|         1|
|      Holden|   Karau|   CA|       6|     37|1551904299|         1|
|      Holden|   Karau|   CA|       4|    153|1552876129|         2|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|         1|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|         2|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|         3|
+------------+--------+-----+--------+-------+----------+----------+



dataw: DataFrame = [firstName: string, lastName: string ... 5 more fields]

In [17]:
dataw.where($"row_number" === 1).show()

24/09/09 20:04:21 INFO CodeGenerator: Code generated in 8.154024 ms
24/09/09 20:04:21 INFO DAGScheduler: Registering RDD 19 (show at cell17.sc:1) as input to shuffle 2
24/09/09 20:04:21 INFO DAGScheduler: Got map stage job 6 (show at cell17.sc:1) with 6 output partitions
24/09/09 20:04:21 INFO DAGScheduler: Final stage: ShuffleMapStage 10 (show at cell17.sc:1)
24/09/09 20:04:21 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:04:21 INFO DAGScheduler: Missing parents: List()
24/09/09 20:04:21 INFO DAGScheduler: Submitting ShuffleMapStage 10 (MapPartitionsRDD[19] at show at cell17.sc:1), which has no missing parents
24/09/09 20:04:21 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 23.2 KiB, free 4.5 GiB)
24/09/09 20:04:21 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 11.1 KiB, free 4.5 GiB)
24/09/09 20:04:21 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on ubuntu:37259 (size: 11.1 KiB, free: 4.

+------------+--------+-----+--------+-------+----------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_number|
+------------+--------+-----+--------+-------+----------+----------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|         1|
|      Holden|   Karau|   CA|       6|     37|1551904299|         1|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|         1|
+------------+--------+-----+--------+-------+----------+----------+



In [18]:
dataw.withColumn("revenue2", nth_value($"revenue", 1).over(windowSpec)).show()

24/09/09 20:04:22 INFO DAGScheduler: Registering RDD 28 (show at cell18.sc:1) as input to shuffle 3
24/09/09 20:04:22 INFO DAGScheduler: Got map stage job 8 (show at cell18.sc:1) with 6 output partitions
24/09/09 20:04:22 INFO DAGScheduler: Final stage: ShuffleMapStage 13 (show at cell18.sc:1)
24/09/09 20:04:22 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:04:22 INFO DAGScheduler: Missing parents: List()
24/09/09 20:04:22 INFO DAGScheduler: Submitting ShuffleMapStage 13 (MapPartitionsRDD[28] at show at cell18.sc:1), which has no missing parents
24/09/09 20:04:22 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 8.9 KiB, free 4.5 GiB)
24/09/09 20:04:22 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 4.7 KiB, free 4.5 GiB)
24/09/09 20:04:22 INFO BlockManagerInfo: Added broadcast_8_piece0 in memory on ubuntu:37259 (size: 4.7 KiB, free: 4.5 GiB)
24/09/09 20:04:22 INFO SparkContext: Created broadcast 8 from br

+------------+--------+-----+--------+-------+----------+----------+--------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_number|revenue2|
+------------+--------+-----+--------+-------+----------+----------+--------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|         1|      91|
|      Holden|   Karau|   CA|       6|     37|1551904299|         1|      37|
|      Holden|   Karau|   CA|       4|    153|1552876129|         2|      37|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|         1|      75|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|         2|      75|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|         3|      75|
+------------+--------+-----+--------+-------+----------+----------+--------+

